In [1]:
# To add a new cell, type '# %%'
# To add a new markdown cell, type '# %% [markdown]'
# %%
from IPython import get_ipython

# %%
get_ipython().system('pip install requests')
get_ipython().system('pip install dict2xml')
get_ipython().system('pip install lxml')
get_ipython().system('pip install googlesearch-python')


# %%
import requests
from lxml import html
from dict2xml import dict2xml
from googlesearch import search


# %%
output_file = "./whatever.xml"
starting_id = 97268 +1 # CHNAGE THIS TO LAST ID IN XML FILE


# %%
def get_unknown_fields(xpath_query):
    name, value = (t.strip() for t in tree.xpath(f"{xpath_query}/text()")[0].split(": "))
    name = name.lower().replace(" ", "_").replace("-", "_").split("(")[0]
    if not value:
        value = tree.xpath(f"{xpath_query}/a/text()")[0].strip()
    return name, value





  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for dict2xml: filename=dict2xml-1.7.0-py3-none-any.whl size=6326 sha256=56cae13ff97796e82d320447e9a9f1888dd10ce7b02d7ecd0433505262a8de9f
  Stored in directory: /root/.cache/pip/wheels/99/2c/cb/8ffb54cf518a693d9157a071f5d428204fa3dc334a45b1abb1
Successfully built dict2xml
     |████████████████████████████████| 115 kB 14.0 MB/s 
     |████████████████████████████████| 61 kB 8.3 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [37]:

# %%
from bs4 import BeautifulSoup
import re
def get_all_text():
    return 

def search_scrape(game_name):
    results =  [r for r in search(game_name, num_results=5) if "embed" not in r]
    link = results[1]
    headers = {
    'User-Agent': 'My User Agent 1.0',
    'From': 'youremail@domain.com'  # This is another valid field
    }

    page = requests.get(link, headers=headers)
    # page = requests.get("https://www.ign.com/articles/fifa-22-review")
    # tree = html.fromstring(page.text)
    # for link in results:
    html_content = page.content
    soup = BeautifulSoup(html_content, 'html.parser')
    text = soup.find_all(text=True)
    output = ''
    # print(set([t.parent.name for t in text]))

    for t in text:
        if t.parent.name not in ['script', 'em', 'style', 'option', 'span', '[document]', 'button']:
            output += '{} '.format(t)
    print(output)

    # tags = soup.findAll("a")
    # all_links = []
    # for a_tag in tags:
    #     href = a_tag.attrs.get("href")
    #     if href[0] == "/":
    #         href = "https://ign.com" + href
    #     all_links.append(href)
    # print(all_links)

    data = soup.findAll('div', attrs={'class':re.compile("article-content")})
    for div in data:
        links = div.findAll('a')
        for a in links:
            print("https://ign.com"+a['href'])


# %%
search_scrape("site:ign.com Fifa")

FIFA 22 Review - IGN IGN Logo Store News Videos Reviews Wikis More Games PlayStation Xbox Nintendo PC Upcoming Best Games Entertainment Movies Television Comics Tech Original Shows Best Picks IGN 25: The Game IGN Themes IGN Boards Follow Us About Us Accessibility Do Not Sell My Personal Information Privacy Policy Site Map Standards Terms of Use Contact Support ©1996-   2022    Ziff Davis, LLC. IGN® is among the federally registered trademarks of IGN Entertainment, Inc. and may only be used with explicit written permission. Search FIFA 22 FIFA 22 Review World in HyperMotion. Loading By  Jordan Oloman  Updated:    1 Oct 2021 3:50 pm Posted:    28 Sep 2021 12:35 am With no alarms and no surprises, EA Sports has pumped out yet another incremental update to its football simulation series.  FIFA 22’s  graphical upgrades and new animation technology make the beautiful game feel better, with goals demanding a more deliberate and rewarding playstyle. Tiny tweaks also improve FIFA 22’s Career Mo

In [ ]:

# %%
# scrapes data from "https://thegamesdb.net/game.php?id={i}" and appends to output_file.xml
with open(output_file, "a", encoding="utf-8") as f:
    f.write("<GAMES>") 
    for i in range(0, 10):
        url = f"https://thegamesdb.net/game.php?id={i}"
        page = requests.get(url)
        tree = html.fromstring(page.text)

        try:
            game_name = tree.xpath("/html/body/div/div[2]/div[2]/div[1]/div/div/div[1]/h1/text()")[0].strip()
            description = tree.xpath("/html/body/div/div[2]/div[2]/div[1]/div/div/div[2]/p[1]/text()")[0].strip()
        except IndexError: continue

        gamesdb_link = url
        esrb_rating = ""
        genres = ""

        for j in range(2,6):
            try:
                name, value = tree.xpath(f"/html/body/div/div[2]/div[2]/div[1]/div/div/div[2]/p[{j}]/text()")[0].split(": ")
                if value:
                        esrb_rating = value if "esrb" in name.lower() else esrb_rating
                        genres = value if "genre" in name.lower() else genres
            except IndexError: continue

        data = {
                    "game_id": i,
                    "game_name": game_name,
                    "description": description,
                    "esrb_rating": esrb_rating,
                    "genres": genres, 
                    "gamesdb_link": gamesdb_link,
                    "ign": search_scrape()
                }

        for k in range(2,10):
            try:
                xpath_query = f"/html/body/div/div[2]/div[1]/div/div/div/div[2]/p[{k}]"
                name, value = get_unknown_fields(xpath_query)
                data[name] = value
            except IndexError: continue

        xml = dict2xml(data, indent=False, wrap="game")
        f.write(xml)
        print(xml, sep="\n----\n")
        # print(i)
    f.write("</GAMES>")


# %%
print("done")